<a href="https://colab.research.google.com/github/MarijaGijic/Julia_kernel_abstractions/blob/main/Morphology_dilation_erosion_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


In [3]:
using Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   2842.3 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 11 seconds. 460 already precompiled.


In [4]:
using Pkg
Pkg.add("CUDA")
Pkg.add("KernelAbstractions")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [63c18a36] + KernelAbstractions v0.9.34
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [5]:
using KernelAbstractions, CUDA, Test

@kernel function dilate_kernel_3D!(output, img_input, struct_element)
    I = @index(Global, Cartesian)
    i, j, k = Tuple(I)
    offset_h = div(size(struct_element)[1], 2)
    offset_w = div(size(struct_element)[2], 2)
    offset_d = div(size(struct_element)[3], 2)

    if k <= size(img_input)[3]
        if i <= size(img_input)[1] && j <= size(img_input)[2]
            result = false

            # Iterate over the structure element
            for m in 1:size(struct_element)[1]
                for n in 1:size(struct_element)[2]
                    for p in 1:size(struct_element)[3]
                        ni = i + m - offset_h - 1
                        nj = j + n - offset_w - 1
                        nk = k + p - offset_d - 1

                        # Check if the indices are within bounds
                        if 1 <= ni <= size(img_input)[1] && 1 <= nj <= size(img_input)[2] && 1 <= nk <= size(img_input)[3]
                            if img_input[ni, nj, nk] == 1 && struct_element[m, n, p] == 1
                                result = true
                                break
                            end
                        end
                    end
                    if result
                        break
                    end
                end
                if result
                    break
                end
            end

            # Set the output pixel to 1 if the result is true, otherwise 0
            @inbounds output[i, j, k] = result ? 1 : 0
        end
    end
end

function dilate!(output, img_input, struct_element)

    backend = get_backend(img_input)
    kernel! = dilate_kernel_3D!(backend)
    kernel!(output, img_input, struct_element, ndrange=size(output))
    return output

end


dilate! (generic function with 1 method)

In [9]:
@kernel function erode_kernel_3D!(output_img, input_img, struct_element)
    I = @index(Global, Cartesian)
    i, j, k = Tuple(I)

    offset_h = div(size(struct_element)[1], 2)
    offset_w = div(size(struct_element)[2], 2)
    offset_d = div(size(struct_element)[3], 2)

    if k <= size(input_img)[3]
      if i <= size(input_img)[1] && j <= size(input_img)[2]
          result = true

          # Iterate over the structure element
          for m in 1:size(struct_element)[1]
              for n in 1:size(struct_element)[2]
                  for p in 1:size(struct_element)[3]
                      ni = i + m - offset_h - 1
                      nj = j + n - offset_w - 1
                      nk = k + p - offset_d - 1

                      # Check if the indices are within bounds
                      if 1 <= ni <= size(input_img)[1] && 1 <= nj <= size(input_img)[2] && 1 <= nk <= size(input_img)[3]
                          if struct_element[m, n, p] == 1 && input_img[ni, nj, nk] == 0
                              result = false
                              break
                          end
                      end
                  end
                  if !result
                      break
                  end
              end
              if !result
                break
              end
          end

            @inbounds output_img[i, j, k] = result ? 1 : 0
        end
    end
end

function erode!(output, img_input, struct_element)

    backend = get_backend(img_input)
    kernel! = erode_kernel_3D!(backend)
    kernel!(output, img_input, struct_element, ndrange=size(output))
    return output

end

erode! (generic function with 1 method)

In [10]:
using CUDA
function testing()
    img_input_dilation = Bool[

        0 0 0 0 0 0 0 0 0 0 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 0 0 0 0 0 0 0 0 0 0;;;

        0 0 0 0 0 0 0 0 0 0 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 0 0 0 0 0 0 0 0 0 0;;;

        0 0 0 0 0 0 0 0 0 0 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 0 0 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 1 1 1 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 0 0 0 1 1 1 1 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 1 1 1 1 1 1 1 0 0 0;
        0 0 0 0 0 0 0 0 0 0 0
        ]

    expected_result_dilation = Bool[
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 0 0;
        1 1 1 1 1 1 1 1 1 0 0;;;

        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 0 0;
        1 1 1 1 1 1 1 1 1 0 0;;;

        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 0 0;
        1 1 1 1 1 1 1 1 1 0 0
        ]

    img_input_erosion = Bool[
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;;;

        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;;;

        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 0 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1
        ]

      expected_result_erosion = Bool[
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;;;

        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;;;

        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 0 0 0 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;
        1 1 1 1 1 1 1 1 1 1 1 1 1;;;
      ]

    # Structuring element
    struct_element = ones(Bool, 3, 3, 3)

    # Output arrays (CPU)
    output_dilation = zeros(Bool, size(expected_result_dilation))
    output_erosion = zeros(Bool, size(expected_result_erosion))

    # Call CPU versions
    dilate!(output_dilation, img_input_dilation, struct_element)
    erode!(output_erosion, img_input_erosion, struct_element)

    # GPU versions
    d_img_input_dilation = CuArray(img_input_dilation)
    d_img_input_erosion = CuArray(img_input_erosion)
    d_struct_element = CuArray(struct_element)

    d_output_dilation = CuArray(zeros(Bool, size(expected_result_dilation)))
    d_output_erosion = CuArray(zeros(Bool, size(expected_result_erosion)))

    dilate!(d_output_dilation, d_img_input_dilation, d_struct_element)
    erode!(d_output_erosion, d_img_input_erosion, d_struct_element)

    # Bring results back from GPU
    gpu_result_dilation = Array(d_output_dilation)
    gpu_result_erosion = Array(d_output_erosion)

    @test gpu_result_dilation == expected_result_dilation
    @test gpu_result_erosion == expected_result_erosion

end

testing (generic function with 1 method)

In [11]:
testing()

Test Passed